In [1]:
import pandas as pd
import numpy as np
import joblib 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 1.从特征重要性最高的前100个特征，

In [2]:
COL_NUM = 100

In [3]:
import time
def timer(func):
    def wrapper(*args,**kwargs):
        start = time.time()
        result = func(*args,**kwargs)
        end = time.time()
        print(func.__name__+'运行时间：',end-start)
        return result
    return wrapper

In [ ]:
f_score = joblib.load('./1.get_raw_fscore/f_score')

input_dir = '../../../preprocess_data_new/'
data_date = joblib.load(input_dir + 'train_ax_date.lz4')[:33465]
data_nodup =joblib.load(input_dir + 'train_ax_nodup.lz4').drop(columns=['id','loan_dt'])[:33465]
data_null = joblib.load(input_dir + 'train_ax_null.lz4')[:33465]
# 为了迁就valid中tag特征append到最后的格式
data_tag = data_nodup[['tag']] 
data_nodup = data_nodup.drop(columns=['tag'])
data = pd.concat([data_date,data_nodup,data_null,data_tag],axis=1)

x = data.fillna(-1).values


In [ ]:
def filter_uniq(df,uniq_num=100):
    '''uniq_num < 100的不要'''
    col_list =[col for col in df.columns if len(df[col].unique())>uniq_num]
    return df[col_list]
def filter_dup(df,percent=80):
    '''重复过多的不要'''
    thre = percent*df.shape[0]
    col_list =[col for col in df.columns if df[col].value_counts().values[0]<thre]
    return df[col_list]
def gen_data(df,col_num=50):
    df_fs = df[f_score[:col_num].index]
    df_uniq = filter_uniq(df_fs,100)
    df_nodup = filter_dup(df_uniq,80)
    return df_nodup

In [ ]:
data_sel = gen_data(data,COL_NUM)

In [44]:
joblib.dump(data_sel,'data_sel.lz4',compress='lz4')

['data_sel.lz4']

### multiply——做乘法交叉

In [24]:
from joblib import Parallel, delayed

def one_cross(df,col_0):
    data_cf = pd.DataFrame()
    for col_1 in df.columns:
        data_cf['{0}*{1}'.format(col_0,col_1)] = (df[col_0]*df[col_1]).values
    return data_cf

@timer
def parallel_cross(df):
    df_list = Parallel(n_jobs=32,verbose=10)(delayed(one_cross)(df,col_0) for col_0 in df.columns)
    return pd.concat(df_list,axis=1)

In [45]:
data_sel = joblib.load('data_sel.lz4')

In [55]:
data_cf = parallel_cross(data_sel)

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   8 tasks      | elapsed:    0.6s
[Parallel(n_jobs=32)]: Done  21 tasks      | elapsed:    1.3s
[Parallel(n_jobs=32)]: Done  32 out of  86 | elapsed:    2.2s remaining:    3.6s
[Parallel(n_jobs=32)]: Done  41 out of  86 | elapsed:    2.7s remaining:    3.0s
[Parallel(n_jobs=32)]: Done  50 out of  86 | elapsed:    3.2s remaining:    2.3s
[Parallel(n_jobs=32)]: Done  59 out of  86 | elapsed:    3.8s remaining:    1.7s
[Parallel(n_jobs=32)]: Done  68 out of  86 | elapsed:    4.3s remaining:    1.1s
[Parallel(n_jobs=32)]: Done  77 out of  86 | elapsed:    4.8s remaining:    0.6s
[Parallel(n_jobs=32)]: Done  86 out of  86 | elapsed:    5.3s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  86 out of  86 | elapsed:    5.3s finished


parallel_cross运行时间： 8.780004262924194


In [59]:
joblib.dump(data_cf,'data_cf_{}.lz4'.format(data_sel.shape[0]),compress='lz4')

['data_cf.lz4']